In [1]:
import wlogs as wl

from datetime import datetime, timedelta

In [2]:
FROM_DATE = datetime(year=2023, month=1, day=23)

In [3]:
guild = wl.GUILD_ID
reports_container = wl.ReportsContainer()
report_data_container = wl.ReportDataContainer()
client = wl.graphQL_client_from_json()

Access token retrieved.


In [4]:
reports = reports_container.extract()

In [5]:
def convert_time(epoch_time_ms):
    return datetime.fromtimestamp(float(epoch_time_ms)/1000)

In [6]:
reports['start'] = reports['startTime'].apply(convert_time)
reports['end'] = reports['endTime'].apply(convert_time)
reports['duration'] = reports['end'] - reports['start']

reports = reports[(reports['segments'] > 0) & (reports['duration'] > timedelta(hours=0)) & (reports['duration'] < timedelta(hours=5)) & (reports['start'] >= FROM_DATE)]

In [7]:
reports_codes = reports['code'].tolist()
reports_codes_stored = report_data_container.reports_codes_stored()
reports_codes_to_request = [code for code in reports_codes if code not in reports_codes_stored]

In [8]:
for code in reports_codes_to_request:
    report_data_json = wl.query_report(code=code, client=client)['data']['reportData']['report']
    report_data_container.store(content=report_data_json, custom_filename=code)